In [19]:
# https://cran.r-project.org/web/packages/penaltyLearning/index.html

In [20]:
# libraries
library(penaltyLearning)
library(data.table)

In [21]:
# choosing training data
dataset <- "detailed"

# features engineering functions
f1 <- function(x) log((x))
f2 <- function(x) log(log(x))

# features

# inputset <- c("length")
# fe       <- list(f2)
# inputset <- c("length", "variance")
# fe       <- list(f2, f1)
inputset <- c("length", "variance", "range_value", "sum_diff")
fe       <- list(f2, f1, f1, f2)

In [22]:
# read data
feature.dt <- fread(paste("../../training_data/", dataset, "/inputs_chosen.csv", sep = ""))
target.dt  <- fread(paste("../../training_data/", dataset, "/outputs.csv", sep = ""))
folds.dt   <- fread(paste("../../training_data/", dataset, "/folds.csv", sep = ""))

In [23]:
# filter censored intervals
target.dt  <- target.dt[!(min.log.lambda == -Inf & max.log.lambda == Inf)]
feature.dt <- feature.dt[sequenceID %in% target.dt$sequenceID]
folds.dt   <- folds.dt[sequenceID %in% target.dt$sequenceID]

In [24]:
# combine
feature.dt.all <- cbind(feature.dt[, c("sequenceID", ..inputset)], folds.dt[, "fold"])
target.dt.all  <- cbind(target.dt, folds.dt[, "fold"])

In [25]:
# apply feature engineering
feature.dt.all[, (inputset) := mapply(function(col, func) func(feature.dt.all[[col]]), col = inputset, func = fe, SIMPLIFY = FALSE)]

In [26]:
# get number of folds
n_folds = dim(unique(folds.dt[,"fold"]))[1]

In [27]:
for (test.fold in 1:n_folds) {
    # get train and test data
    feature.mat.train <- as.matrix(feature.dt.all[feature.dt.all$fold != test.fold, ..inputset])
    feature.mat.test  <- as.matrix(feature.dt.all[feature.dt.all$fold == test.fold, ..inputset])
    target.mat.train  <- as.matrix(target.dt.all[target.dt.all$fold != test.fold, c("min.log.lambda", "max.log.lambda")])

    # # double check the size of train and test set
    # print(paste("Size of train matrix is:", dim(feature.mat.train)[1], "rows and", dim(feature.mat.train)[2], "columns"))
    # print(paste("Size of test matrix is:", dim(feature.mat.test)[1], "rows and", dim(feature.mat.test)[2], "columns"))
    
    # train model
    fit <- IntervalRegressionUnregularized(
        feature.mat = feature.mat.train,
        target.mat = target.mat.train)
    
    # get prediction from test set
    target.mat.pred <- fit$predict(feature.mat.test)
    colnames(target.mat.pred) <- "llda"

    # save prediction to csv
    prediction <- data.frame(as.matrix(feature.dt.all[feature.dt.all$fold == test.fold, "sequenceID"]), target.mat.pred)
    write.csv(prediction, file = paste("predictions/", dataset, test.fold, length(inputset), "csv", sep = "."), row.names = FALSE)
}